# Laboratorio 8 Anomalias

----

Santiago Pereira 22318

Nancy Mazariegos 22513

----


In [6]:
import numpy as np, pandas as pd
from sklearn.datasets import fetch_covtype

data = fetch_covtype(as_frame=True)
X = data.data
y_cover = data.target

print(X.shape, y_cover.unique())


(581012, 54) [5 2 1 7 3 6 4]


In [10]:
import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split

y_anom = (y_cover != 2).astype(int)  

mask_norm = (y_anom == 0)
X_norm, X_anom = X[mask_norm], X[~mask_norm]

X_train_norm, X_val_norm = train_test_split(
    X_norm, test_size=0.2, random_state=42, shuffle=True
)

X_test = pd.concat([X_val_norm, X_anom], axis=0)
y_test = pd.concat([
    pd.Series(0, index=X_val_norm.index),
    pd.Series(1, index=X_anom.index)
])
X_train_norm.shape, X_val_norm.shape, X_test.shape, y_test.value_counts()


((226640, 54),
 (56661, 54),
 (354372, 54),
 1    297711
 0     56661
 Name: count, dtype: int64)

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

num_cols = X.columns[:10]

preprocess = ColumnTransformer(
    transformers=[('scale', StandardScaler(), num_cols)],
    remainder='passthrough'  
)

X_train = preprocess.fit_transform(X_train_norm)
X_val   = preprocess.transform(X_val_norm)
X_test_ = preprocess.transform(X_test)

X_train.shape, X_val.shape, X_test_.shape


((226640, 54), (56661, 54), (354372, 54))

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

tf.random.set_seed(42)

input_dim = X_train.shape[1]

inp = layers.Input(shape=(input_dim,))
x = layers.Dense(128, activation='relu')(inp)
x = layers.Dense(64, activation='relu')(x)
encoded = layers.Dense(32, activation='relu')(x)
x = layers.Dense(64, activation='relu')(encoded)
x = layers.Dense(128, activation='relu')(x)
out = layers.Dense(input_dim, activation='linear')(x)

autoencoder = models.Model(inp, out)
autoencoder.compile(optimizer='adam', loss='mse')

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
hist = autoencoder.fit(
    X_train, X_train,
    validation_data=(X_val, X_val),
    epochs=100, batch_size=512,
    callbacks=[es], verbose=1
)


ModuleNotFoundError: No module named 'tensorflow'